In [122]:
import os
import pandas as pd

data_df = pd.DataFrame(columns=['text','Intent'])
filenames = list()
intervention_subgraphs = list()
for directory, subdirectories, files in os.walk('C:/Users/Tamir/Desktop/HSE_Project/excel_files/'):
    for file in files:
        filenames.append(os.path.join(directory,file))
for filename in filenames:
    if (filename[len(filename) - 22] == '~'):
        continue
    filename = filename.replace(" ", " ")
    post_df = pd.read_excel(filename)
    
        #fix to add columns if they are not in the xlsx file
    if 'attachments' not in post_df.columns:
        post_df['attachments'] = ''
    if 'reply_to_comment' not in post_df.columns:
        post_df['reply_to_comment'] = 0
    if 'reply_to_user' not in post_df.columns:
        post_df['reply_to_user'] = 0
        
    flex_df = post_df.drop(columns=['attachments','date','from_id','id','reply_to_comment','reply_to_user','Content','Intervention'])
    data_df = data_df.append(flex_df,ignore_index = True)

In [123]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
def clean_text(text, remove_stopwords=True):
    """
    Removes stop words from the given text
    :param text: string with the text
    :param remove_stopwords: flag to enable removal of stop words
    :return: set of tokens without any stop words
    """
    text = text.lower()
    tokens = tokenizer.tokenize(text)

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('russian'))
        tokens = [w for w in tokens if w not in stops]

    return tokens

delete_index = list()

for index,row in data_df.iterrows():
    if ( pd.isnull(data_df['text'].loc[index]) or pd.isnull(data_df['Intent'].loc[index]) 
              or (data_df['Intent'].loc[index] == ' -' ) ):
        delete_index.append(index)        
    else:
        data_df['text'].loc[index] = clean_text(data_df['text'].loc[index])

data_df = data_df.drop(index = delete_index)
data_df = data_df.reset_index(drop=True)

for index,row in data_df.iterrows():
    data_df['Intent'].loc[index] = data_df['Intent'].loc[index][0]
    if(data_df['Intent'].loc[index] == 'E'):
        print('change in ' + str(index))
        data_df['Intent'].loc[index] = 'Е'

change in 3158


In [140]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

documents = [TaggedDocument(data_df['text'].loc[i], data_df['Intent'].loc[i]) for i in range(len(data_df))]
model = Doc2Vec(documents,dm=1, vector_size=10, window=4, min_count=1,epochs=5, workers=4)

In [141]:
model.most_similar('путин')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('русских', 0.9985089898109436),
 ('которые', 0.9984912872314453),
 ('лет', 0.9984446167945862),
 ('хотя', 0.99834805727005),
 ('нато', 0.9983335137367249),
 ('сергей', 0.9982507228851318),
 ('войны', 0.9979708194732666),
 ('вообще', 0.9979184865951538),
 ('тебе', 0.9978156089782715),
 ('будут', 0.9978112578392029)]

In [125]:
data_df['Intent'].unique()

array(['Ц', 'Н', 'Ш', 'Ж', 'И', 'Ч', 'Т', 'Щ', 'А', 'Г', 'У', 'Х', 'Б',
       'Д', 'Л', 'В', 'Е', 'П', 'К', 'С', 'М', 'Ф', 'З', 'Р', 'О'],
      dtype=object)

In [143]:
tokens = "ага держи карман шире".split()

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vector])
sims

D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('У', 0.7306615114212036),
 ('А', 0.7306163311004639),
 ('Г', 0.7295863032341003),
 ('Ж', 0.7294228672981262),
 ('Б', 0.7290605306625366),
 ('Ц', 0.7276159524917603),
 ('В', 0.7276017665863037),
 ('С', 0.7266857624053955),
 ('М', 0.7257146835327148),
 ('Х', 0.7248767018318176)]